[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/H-IAAC/d3vis_ipynb/blob/main/examples/web_widget.ipynb)

In [ ]:
from d3vis_ipynb import CustomWidget, Embedding
import traitlets

# Import Datasets

In [ ]:
import statsmodels.api as sm
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
affairs = sm.datasets.get_rdataset('Affairs', 'AER').data
affairsData = affairs.to_dict(orient='records')
affairs.head()

# CustomWidget

Generates a widget using JavaScript code from a external source. It's an encapsulation of the anywidget library to generate a widget without hanving to implement the render function.

Functions:
- **createWidgetFromUrl**: Creates a widget using a url. Parameters:
    - **widgetCall**: a string that represents how the JavaScriptFunction should be called
    - **varList**: a list of variables used by the widget. The variable *ElementId* is necessary  for the widget to function with the *Embedding* tool
    - **updatableVars**: a subset of varList, only the variables that can re-render the widget when changed
    - **jsUrl**: the url from which the JavaScript code come from
 
- **createWidgetFromLocalFile**: Creates a widget using a url. Parameters:
    - **widgetCall**: a string that represents how the JavaScriptFunction should be called
    - **varList**: a list of variables used by the widget. The variable *ElementId* is necessary  for the widget to function with the *Embedding* tool
    - **updatableVars**: a subset of varList, only the variables that can re-render the widget when changed
    - **filePath**: the file path from which the JavaScript code come from

CustomWidget rules:
- a class encapluating CustomWidget must be defined with the following variables:
    - **_esm**: must contain the JavaScript code
    - **_css**: must contain the css code (optional)
- variables with the same names as in *varList* must be defined with traitlets
- one of the parameters of the main JavaScript function must be *element*, this parameter is where the html/js contents must be generated

The example bellow shows the same implementation as shown in the example *widgets.ipynb* but using CustomWidget instead:

## Barplot

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/barplot.js"

class CustomBarplot(CustomWidget):
    _esm = CustomWidget.createWidgetFromUrl(widgetCall="barplot(data, x, y, hue, element, width, height, margin)",
                               varList=["data", "x", "y", "hue", "elementId"],
                               updatableVars=["data", "x", "y", "hue", "elementId"],
                               jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    y = traitlets.Unicode().tag(sync=True)
    hue = traitlets.Unicode().tag(sync=True)
    elementId = traitlets.Unicode().tag(sync=True)

barplot = CustomBarplot(data=affairsData, x='education', y='rating')

## HistogramPlot

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/histogramplot.js"

class CustomHistogramplot(CustomWidget):
    _esm = CustomWidget.createWidgetFromUrl(widgetCall="histogramplot(data, x, start, end, element, width, height, margin)",
                               varList=["data", "x", "start", "end", "elementId"],
                               updatableVars=["data", "x", "start", "end", "elementId"],
                               jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    start = traitlets.Float().tag(sync=True)
    end = traitlets.Float().tag(sync=True)
    elementId = traitlets.Unicode().tag(sync=True)

histplot = CustomHistogramplot(data=affairsData, x='education')

## ScatterPlot

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/scatterplot.js"

class CustomScatterplot(CustomWidget):
    _esm = CustomWidget.createWidgetFromUrl(widgetCall="scatterplot(data, x, y, hue, element, setValue, setSelectedValues, width, height, margin)",
                               varList=["data", "x", "y", "hue", "elementId", "clickedValue", "selectedValues"],
                               updatableVars=["data", "x", "y", "hue", "elementId"],
                               jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    y = traitlets.Unicode().tag(sync=True)
    hue = traitlets.Unicode().tag(sync=True)
    elementId = traitlets.Unicode().tag(sync=True)
    clickedValue = traitlets.Unicode().tag(sync=True)
    selectedValues = traitlets.List([]).tag(sync=True)

    def on_select_values(self, callback):
        self.observe(callback, names=["selectedValues"])

    def on_click_value(self, callback):
        self.observe(callback, names=["clickedValue"])

scatterplot = CustomScatterplot(data = affairsData, x = 'age', y = 'yearsmarried', hue = "children")

## RangeSlider

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/rangeslider.js"

from d3vis_ipynb import CustomWidget
import traitlets
class CustomRangeSlider(CustomWidget):
    _esm = CustomWidget.createWidgetFromUrl(widgetCall="rangeslider(data, variable, step, description, minValue, maxValue, setValues, element, margin)",
                               varList=["data", "variable", "step", "description", "minValue", "maxValue", "elementId"],
                               updatableVars=["data", "variable", "step", "description", "elementId"],
                               jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    variable = traitlets.Unicode().tag(sync=True)
    step = traitlets.Float().tag(sync=True)
    description = traitlets.Unicode().tag(sync=True)
    minValue = traitlets.Float().tag(sync=True)
    maxValue = traitlets.Float().tag(sync=True)
    elementId = traitlets.Unicode().tag(sync=True)

    def on_drag(self, callback):
        self.observe(callback, names=["minValue", "maxValue"])

rangeSlider = CustomRangeSlider(data=affairsData, variable="age", step=0.1, description="Age:")

## Embedding

In [ ]:
def on_selected(values):
    barplot.data = scatterplot.selectedValues
    histplot.data = scatterplot.selectedValues
    print(scatterplot.selectedValues)
    
scatterplot.on_select_values(on_selected)

def on_values_changed(values):
    newData = affairs[affairs["age"].between(
        rangeSlider.minValue, rangeSlider.maxValue)].to_dict(orient='records')
    scatterplot.data = newData

rangeSlider.on_drag(on_values_changed)

matrix = [
    [3, 3, 1, 1], 
    [3, 3, 1, 1],
    [3, 3, 2, 2],
    [4, 4, 2, 2]
]
embed = Embedding(matrix, style="neumorphism")
embed.add(barplot, 1)
embed.add(histplot, 2)
embed.add(scatterplot, 3)
embed.add(rangeSlider, 4)

embed